# Spoint tutorial: Deconvolution of cell types composition on human brain Visium dataset
# 细胞类型组成的反卷积在人脑 Visium 数据集上

Dataset: Visium slices of human dorsolateral prefrontal cortex ([here](https://zenodo.org/record/8167488)).

In [1]:
from SPACEL.setting import set_environ_seed
set_environ_seed()
from SPACEL import Spoint
import scanpy as sc
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.serif'] = ['Arial']
sc.settings.set_figure_params(dpi=50,dpi_save=300,facecolor='white',fontsize=10,vector_friendly=True,figsize=(3,3))
sc.settings.verbosity = 3

Setting environment seed: 42


Global seed set to 0


## Load spatial transcriptomics data

输入数据是 anndata objects 存储 scRNA-seq 和 ST 的 raw counts。scRNA-seq anndata 必须在 `.obs` 中具有细胞类型注释。

In [2]:
sc_ad = sc.read_h5ad('../data/visium_human_DLPFC/human_MTG_snrna_norm_by_exon.h5ad')
st_ad = sc.read_h5ad('../data/visium_human_DLPFC/human_DLPFC_spatial_151676.h5ad')

sc.pp.filter_genes(st_ad,min_cells=1)
sc.pp.filter_genes(sc_ad,min_cells=1)
sc.pp.filter_cells(st_ad,min_genes=1)
sc.pp.filter_cells(sc_ad,min_genes=1)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '../data/visium_human_DLPFC/human_MTG_snrna_norm_by_exon.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Initialize the Spoint model

在此步骤中，我们使用 scRNA-seq 和 ST 的 anndata 对象作为输入来初始化 Spoint 模型。`celltype_key` 参数表示 scRNA-seq anndata 对象的 `.obs` 属性中细胞类型注释的列名称。`sm_size` 参数控制模拟 spots 的数量，拥有足够的 `sm_size` 对于准确预测非常重要。但需要注意的是，增加 `sm_size` 也会增加模拟和训练时间。一般来说，我们建议将 `sm_size` 设置为大于 100,000 的值。

In [ ]:
spoint_model = Spoint.init_model(sc_ad,st_ad,celltype_key='cluster_label',deg_method='t-test',sm_size=100000,use_gpu=True)

# Training the Spoint model

在这里，我们训练模型以获得细胞类型反卷积的最佳模型。`max_steps` 参数表示训练过程中的最大步长。如果 `early_stop` 参数设置为 `True`，如果模型已经收敛，模型将在达到最大步数之前停止训练。

In [ ]:
spoint_model.train(max_steps=5000, batch_size=1024)

## Visualization results

然后，我们利用训练好的模型来预测空间转录组数据中每个 spot 的细胞类型组成。该预测将生成一个 `DataFrame` 对象，其中每一行对应于空间转录组数据中的一个点，每一列代表单细胞 RNA-seq 数据中的一种细胞类型，每个条目表示特定细胞类型在一个 spot 中所占的比例。此外，我们还可以获得空间转录组数据的 anndata 对象，并将反卷积结果嵌入到 `.obs` 属性中。

In [ ]:
pre = spoint_model.deconv_spatial()

In [ ]:
st_ad = spoint.st_ad

我们可以在ST切片中看到反卷积的结果。

In [ ]:
sc.pl.embedding(st_ad,color=pre.columns[:20],basis='spatial',ncols=5)